<h3><center>Detecting Fake News</center></h3>   

    
<center>Sam Loyd</center>
<center>July 2021</center> 

In [1]:
# Required Modules
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from imblearn.over_sampling import *
from imblearn.combine import SMOTETomek
from collections import Counter
from pycaret.classification import *
from sklearn import metrics
from sklearn.impute import KNNImputer
import pickle
from sklearn.preprocessing import StandardScaler
import os
import neattext as nt

In [2]:
# Pull in data for stroke data 
true_df = pd.read_csv("data\\True.csv")
fake_df = pd.read_csv("data\\Fake.csv")

In [3]:
# Set status
true_df['fake']=0
fake_df['fake']=1

In [4]:
frames = [true_df, fake_df]

In [5]:
fake_news_df = pd.concat(frames)


In [6]:
fake_news_df

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [7]:
fake_news_df = fake_news_df.reset_index(drop=True)

In [8]:
fake_news_df

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [9]:
fake_news_df.dtypes

title      object
text       object
subject    object
date       object
fake        int64
dtype: object

In [10]:
# Remove entries with bogus dates
tmp_df = fake_news_df[fake_news_df.date.apply(lambda x: len(str(x))<=30)] 

In [11]:
fake_news_df = tmp_df

In [12]:
fake_news_df = fake_news_df.reset_index(drop='True')

In [13]:
# convert the 'Date' column to datetime format
fake_news_df['date']= pd.to_datetime(fake_news_df['date'])

In [14]:
fake_news_df

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,0
...,...,...,...,...,...
44883,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,2016-01-16,1
44884,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,2016-01-16,1
44885,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,2016-01-15,1
44886,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,2016-01-14,1


In [15]:
# Apply profile and create html report
# Commented out for performance
# prof = ProfileReport(fake_news_df,title="Pandas Profiling Report - Fake News Data", explorative=True)
# prof.to_file(output_file='fakenews-pandas-profile-report.html')

# Please see stroke-pandas-profile-report.html for a full analysis 

In [16]:
# Remove 196 duplicates
fake_news_df.drop_duplicates(keep='first', inplace=True)

In [17]:
# Function to clean up text 
# Replaces all of these and then some with level = deep ...

#neattxtpr = nt.remove_puncts(neattxtpr)
#neattxtpr = nt.remove_stopwords(neattxtpr)
#neattxtpr = nt.remove_html_tags(neattxtpr)
#neattxtpr = nt.remove_special_characters(neattxtpr)
#neattxtpr = nt.remove_emojis(neattxtpr)
#neattxtpr = nt.fix_contractions(neattxtpr)

def preproc(neattxt):
    neattxtpr = nt.normalize(neattxt,level='deep')
    neattxtpr = nt.remove_stopwords(neattxtpr)

    return neattxtpr

In [18]:
# preprocess title
fake_news_df['title_cl'] = fake_news_df['title'].apply(preproc)

In [19]:
fake_news_df['text_cl'] = fake_news_df['text'].apply(preproc)

In [20]:
fake_news_df

,title,text,subject,date,fake,title_cl,text_cl
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,0,budget fight looms republicans flip fiscal script,washington reuters head conservative republica...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,0,military accept transgender recruits monday pe...,washington reuters transgender people allowed ...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,0,senior republican senator let mr mueller job,washington reuters special counsel investigati...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,0,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geor...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,0,trump wants postal service charge amazon shipm...,seattlewashington reuters president donald tru...
...,...,...,...,...,...,...,...
44883,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,2016-01-16,1,mcpain john mccain furious iran treated sailors,century wire says reported earlier week unlike...
44884,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,2016-01-16,1,justice yahoo settles email privacy classactio...,century wire says familiar theme dispute chang...
44885,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,2016-01-15,1,sunnistan allied safe zone plan territorial bo...,patrick henningsen century wireremember obama ...
44886,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,2016-01-14,1,blow million al jazeera america finally calls ...,century wire says al jazeera america history b...


In [21]:
# Verify types
fake_news_df.dtypes

title               object
text                object
subject             object
date        datetime64[ns]
fake                 int64
title_cl            object
text_cl             object
dtype: object

In [22]:
# Drop subject as it must be future data to be almost 100% correlated to fake
clean_fake_news_df = fake_news_df.drop(['subject','title','text'], axis=1)

In [23]:
# Take a look 
clean_fake_news_df

,date,fake,title_cl,text_cl
0,2017-12-31,0,budget fight looms republicans flip fiscal script,washington reuters head conservative republica...
1,2017-12-29,0,military accept transgender recruits monday pe...,washington reuters transgender people allowed ...
2,2017-12-31,0,senior republican senator let mr mueller job,washington reuters special counsel investigati...
3,2017-12-30,0,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geor...
4,2017-12-29,0,trump wants postal service charge amazon shipm...,seattlewashington reuters president donald tru...
...,...,...,...,...
44883,2016-01-16,1,mcpain john mccain furious iran treated sailors,century wire says reported earlier week unlike...
44884,2016-01-16,1,justice yahoo settles email privacy classactio...,century wire says familiar theme dispute chang...
44885,2016-01-15,1,sunnistan allied safe zone plan territorial bo...,patrick henningsen century wireremember obama ...
44886,2016-01-14,1,blow million al jazeera america finally calls ...,century wire says al jazeera america history b...


In [24]:
# Get year, month, day, julia and day of week

In [25]:
clean_fake_news_df['year'] = pd.DatetimeIndex(clean_fake_news_df['date']).year

In [26]:
clean_fake_news_df['month'] = pd.DatetimeIndex(clean_fake_news_df['date']).month

In [27]:
clean_fake_news_df['day'] = pd.DatetimeIndex(clean_fake_news_df['date']).day

In [28]:
clean_fake_news_df['julia'] = pd.DatetimeIndex(clean_fake_news_df['date']).to_julian_date()

In [29]:
clean_fake_news_df['day'] = pd.DatetimeIndex(clean_fake_news_df['date']).day

In [30]:
clean_fake_news_df['dayofweek'] = clean_fake_news_df['date'].dt.day_name()

In [31]:
# Check stats
clean_fake_news_df.describe()

,fake,year,month,day,julia
count,44679.000000,44679.000000,44679.000000,44679.000000,4.467900e+04
mean,0.525258,2016.521565,7.208241,15.560823,2.457782e+06
std,0.499367,0.601693,3.516935,8.677324,2.530573e+02
min,0.000000,2015.000000,1.000000,1.000000,2.457112e+06
25%,0.000000,2016.000000,4.000000,8.000000,2.457578e+06
50%,1.000000,2017.000000,8.000000,15.000000,2.457816e+06
75%,1.000000,2017.000000,10.000000,23.000000,2.458020e+06
max,1.000000,2018.000000,12.000000,31.000000,2.458168e+06


In [32]:
# Take a look
clean_fake_news_df

,date,fake,title_cl,text_cl,year,month,day,julia,dayofweek
0,2017-12-31,0,budget fight looms republicans flip fiscal script,washington reuters head conservative republica...,2017,12,31,2458118.5,Sunday
1,2017-12-29,0,military accept transgender recruits monday pe...,washington reuters transgender people allowed ...,2017,12,29,2458116.5,Friday
2,2017-12-31,0,senior republican senator let mr mueller job,washington reuters special counsel investigati...,2017,12,31,2458118.5,Sunday
3,2017-12-30,0,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geor...,2017,12,30,2458117.5,Saturday
4,2017-12-29,0,trump wants postal service charge amazon shipm...,seattlewashington reuters president donald tru...,2017,12,29,2458116.5,Friday
...,...,...,...,...,...,...,...,...,...
44883,2016-01-16,1,mcpain john mccain furious iran treated sailors,century wire says reported earlier week unlike...,2016,1,16,2457403.5,Saturday
44884,2016-01-16,1,justice yahoo settles email privacy classactio...,century wire says familiar theme dispute chang...,2016,1,16,2457403.5,Saturday
44885,2016-01-15,1,sunnistan allied safe zone plan territorial bo...,patrick henningsen century wireremember obama ...,2016,1,15,2457402.5,Friday
44886,2016-01-14,1,blow million al jazeera america finally calls ...,century wire says al jazeera america history b...,2016,1,14,2457401.5,Thursday


In [37]:
one_hot = pd.get_dummies(clean_fake_news_df.dayofweek)

In [40]:
clean_fake_news_df = clean_fake_news_df.drop('dayofweek',axis = 1)
clean_fake_news_df = clean_fake_news_df.join(one_hot)

In [41]:
# final clean up before passing ...
clean_fake_news_df.reset_index(drop=True, inplace=True)

In [42]:
# Apply profile and create html report
# Commented out for performance
prof = ProfileReport(clean_fake_news_df,title="Pandas Profiling Report - Cleaned Fake News Data", explorative=True)
prof.to_file(output_file='clean_fakenews-pandas-profile-report.html')

# Please see stroke-pandas-profile-report.html for a full analysis 